In [ ]:
import pandas as pd
base = pd.read_csv('house_prices.csv')
base.head()

In [ ]:
base.columns

In [ ]:
colunas_usadas = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']

In [ ]:
colunas_usadas

In [ ]:
base = pd.read_csv('house_prices.csv', usecols = colunas_usadas)

In [ ]:
base.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']] = scaler_x.fit_transform(base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']])

In [ ]:
base.head()

In [ ]:
scaler_y = MinMaxScaler()
base[['price']] = scaler_y.fit_transform(base[['price']])

In [ ]:
base.head()

In [ ]:
X = base.drop('price', axis = 1)
y = base.price

In [ ]:
X.head()

In [ ]:
type(X)

In [ ]:
y.head()

In [ ]:
type(y)

In [ ]:
previsores_colunas = colunas_usadas[1:17]
previsores_colunas

In [ ]:
import tensorflow as tf

In [ ]:
colunas = [tf.feature_column.numeric_column(key = c) for c in previsores_colunas]

In [ ]:
print(colunas[10])

In [ ]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3)

In [ ]:
X_treinamento.shape

In [ ]:
y_treinamento.shape

In [ ]:
X_teste.shape

In [ ]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento, y = y_treinamento,
                                                        batch_size = 32, num_epochs = None, shuffle = True)

In [ ]:
funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_teste, y = y_teste,
                                                   batch_size = 32, num_epochs = 10000, shuffle = False)

In [ ]:
regressor = tf.estimator.LinearRegressor(feature_columns=colunas)

In [ ]:
regressor.train(input_fn=funcao_treinamento, steps = 10000)

In [ ]:
metricas_treinamento = regressor.evaluate(input_fn=funcao_treinamento, steps = 10000)

In [ ]:
metricas_teste = regressor.evaluate(input_fn=funcao_teste, steps = 10000)

In [ ]:
metricas_treinamento

In [ ]:
metricas_teste

In [ ]:
funcao_previsao = tf.estimator.inputs.pandas_input_fn(x = X_teste, shuffle = False)

In [ ]:
previsoes = regressor.predict(input_fn=funcao_previsao)

In [ ]:
list(previsoes)

In [ ]:
valores_previsoes = []
for p in regressor.predict(input_fn=funcao_previsao):
    valores_previsoes.append(p['predictions'])

In [ ]:
valores_previsoes

In [ ]:
import numpy as np
valores_previsoes = np.asarray(valores_previsoes).reshape(-1,1)

In [ ]:
valores_previsoes

In [ ]:
valores_previsoes = scaler_y.inverse_transform(valores_previsoes)
valores_previsoes

In [ ]:
y_teste

In [ ]:
y_teste.shape

In [ ]:
y_teste2 = y_teste.values.reshape(-1,1)
y_teste2.shape

In [ ]:
y_teste2 = scaler_y.inverse_transform(y_teste2)

In [ ]:
y_teste2

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_teste2, valores_previsoes)

In [ ]:
mae